<a href="https://colab.research.google.com/github/adimis-ai/Multiple-Document-Abstractive-Long-Form-Q-A/blob/main/Multiple_Document_Abstractive_Long_Form_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-i6yzr_3y/farm-haystack_421a0a01d1fd459d9ac1f357b12b90d9
  Resolved https://github.com/deepset-ai/haystack.git to commit db6e5754cd71dd822f466bebe3882dc61de00c44
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━

  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-i6yzr_3y/farm-haystack_421a0a01d1fd459d9ac1f357b12b90d9


In [ ]:
# setup logs
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
#import faiss document store and assign it to document_store variable
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat",return_embedding=True)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [ ]:
#imports for working with data
from haystack.utils import convert_files_to_docs, clean_wiki_text

# Let's first get some files that we want to use
doc_dir = "gdrive/MyDrive/wiki_gameofthrones_txt1/"

# Convert files to dicts
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/130_Game_of_Thrones_title_sequence.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/102_The_Princess_and_the_Queen.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/347_Game_of_Thrones__season_2_.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/346_Ygritte.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/37_Joffrey_Baratheon.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/310_Mother_s_Mercy.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/99_A_Song_of_Ass_and_Fire.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/311_Game_of_Thrones__season_7_.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/224_Th

Writing Documents:   0%|          | 0/2497 [00:00<?, ?it/s]

In [ ]:
#import dense passage retriever
from haystack.nodes import DensePassageRetriever

# retriever takes in query & convert it to dense vectors using pretrained model for generator
# search & fetches relevant passage(context) from the document store & convert it to dense vectors using pretrained model for generator
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)

document_store.update_embeddings(retriever)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model:Auto-detected model language: english


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.document_stores.faiss:Updating embeddings for 2357 docs...


Updating Embedding:   0%|          | 0/2357 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/2368 [00:00<?, ? Docs/s]

In [ ]:
# we now initalize our reader/generator.
from haystack.nodes import Seq2SeqGenerator

# Here we use a Seq2SeqGenerator with the vblagoje/bart_lfqa model
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
# With a Haystack Pipeline you can stick together your building blocks to a search pipeline. 
# Under the hood, Pipelines are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases
from haystack.pipelines import GenerativeQAPipeline

#GenerativeQAPipeline that combines a retriever and a reader/generator to answer our questions. 
pipe = GenerativeQAPipeline(generator, retriever)

In [ ]:
from haystack.utils import print_answers

def ans(QUERY):
  output = pipe.run(query=QUERY, params={"Retriever": {"top_k": 3}})
  cleaned_output = print_answers(output, details="minimum")
  return cleaned_output

In [ ]:
Question = input()
Answer = ans(Question)

what is the main story of game of thrones

Query: what is the main story of game of thrones
Answers:
[   {   'answer': 'The main story of Game of Thrones is the politics of the '
                  'Seven Kingdoms of Westeros. The series is based on George '
                  "R.R. Martin's *A Song of Ice and Fire* series of books, "
                  'which were published in 1996. The books are set in a '
                  'fictional world called Westeros, which is one of the '
                  'largest and most populous countries in the world. In the '
                  'books, there are three main dynasties in Westeros: the '
                  'Lannisters, the Targaryens, and the Dothraki. The '
                  'Lannisters are the ruling family of the world, and they '
                  'have a lot of power and influence. The Targariesons are a '
                  'group of noblemen who are descended from King Robert '
                  'Baratheon of the House of Lannister, who wa